In [1]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import r2_score
import pandas as pd
import os

os.environ["CUDA_VISIBLE_DEVICES"]="-1"

# 데이터를 불러온 후 Normalization

In [1]:
data = np.loadtxt("ACL nomalization data.txt")
#data = np.Loadtxt("acl data.txt")

#X_origin = data[:,0:4]
#Y_origin = data[:,4]

#X_normdata = (X_origin-np.mean(X_origin)-np.min(X_origin))/(np.max(X_origin)-np.min(X_origin))
#Y_normdata = (Y_origin-np.mean(Y_origin)-min(Y_origin))/(max(Y_origin)-min(Y_origin))

#for i in range(0,4,1):
#    X_normdata[:,i] = (X_origin[:,i]-np.mean(X_origin[:,i]))/(np.max(X_origin[:,i])-np.min(X_origin[:,i]))

#Y_normdata = (Y_normdata-np.mean(Y_normdata))/(max(Y_normdata)-min(Y_normdata))

#X_data = data[:,0:4]
#Y_data = data[:,4]

X_normdata = data[:,0:4]
Y_normdata = data[:,4]

X_train = X_normdata[0:21,0:4]
Y_train = Y_normdata[0:2,]

X_test = X_normdata[21:,0:4]
Y_test = Y_normdata[21:,]

dataframeX_origin = pd.DataFrame(X_normdata)
dataframeY_origin = pd.DataFrame(Y_normdata)

table1 = pd.concat([dataframeX_origin, dataframeY_origin], axis = 1)
table1.columns = ['Pow.', 'Press.', 'C3H6', 'N2', 'Thick.']
table1
print()

NameError: name 'pd' is not defined

# 정규화 된 데이터를 표로 확인

In [3]:
dataframeX_train=pd.DataFrame(X_train) #Include X to dfX, and make it to table type.
dataframeY_train=pd.DataFrame(Y_train)

dataframeX_test=pd.DataFrame(X_test)
dataframeY_test=pd.DataFrame(Y_test)

table2=pd.concat([dataframeX_train, dataframeY_train], axis=1) #If axis = 1, then concat to right side. Otherwise (axis = 0), then concat to bottom side.
                                    #Also, if they have sharing indexes or columns, then it will be write on the same indexes or columns.
                                    #But, if it doesn't, then it will be write on the different indexes or columns, 
                                    #and it will be appear as NaN in the not shared parameters.
            
table3=pd.concat([dataframeX_test, dataframeY_test], axis=1)
table2.columns = ['Pow.', 'Press.', 'C3H6', 'N2', 'Thick.'] #P
table3.columns = ['Pow.', 'Press.', 'C3H6', 'N2', 'Thick.'] #P

print(table2)
print(table3)

    Pow.  Press.  C3H6   N2    Thick.
0    0.0     0.0   0.5  0.5 -0.114209
1    0.0    -0.5   0.0  0.5 -0.406936
2   -0.5    -0.5   0.0  0.0 -0.355118
3   -0.5     0.0   0.5  0.0 -0.150572
4    0.0     0.0   0.0  0.0  0.002155
5    0.5     0.0   0.0  0.5 -0.134209
6    0.0     0.0  -0.5  0.5  0.175791
7    0.5    -0.5   0.0  0.0 -0.268754
8    0.0     0.0  -0.5 -0.5  0.182155
9    0.0    -0.5  -0.5  0.0 -0.128754
10   0.0     0.0   0.5 -0.5 -0.014209
11  -0.5     0.0   0.0 -0.5  0.123064
12   0.5     0.5   0.0  0.0  0.593064
13   0.0    -0.5   0.5  0.0 -0.317845
14  -0.5     0.5   0.0  0.0  0.241246
15   0.5     0.0   0.0 -0.5 -0.069663
16  -0.5     0.0  -0.5  0.0  0.169428
17   0.0     0.5   0.5  0.0  0.061246
18   0.0     0.5   0.0 -0.5  0.210337
19   0.0     0.0   0.0  0.0 -0.086936
20   0.5     0.0   0.5  0.0 -0.146936
   Pow.  Press.  C3H6   N2    Thick.
0   0.0     0.5  -0.5  0.0  0.446700
1   0.0    -0.5   0.0 -0.5 -0.307845
2   0.5     0.0  -0.5  0.0  0.233064
3  -0.5     0.0 

In [4]:
tf.set_random_seed(777)  # for reproducibility
learning_rate = 0.00001

X = tf.placeholder(tf.float32, [None, 4])
Y = tf.placeholder(tf.float32, [None,])

W1 = tf.Variable(tf.random_normal([4, 30],stddev=0.01), name='weight1')
b1 = tf.Variable(tf.random_normal([30], stddev=0.01), name='bias1')
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([30, 30],stddev=0.01), name='weight2')
b2 = tf.Variable(tf.random_normal([30], stddev=0.01), name='bias2')
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([30, 30],stddev=0.01), name='weight2')
b3 = tf.Variable(tf.random_normal([30], stddev=0.01), name='bias2')
layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)

W4 = tf.Variable(tf.random_normal([30, 1],stddev = 0.01), name='weight3')
b4 = tf.Variable(tf.random_normal([1], stddev=0.01), name='bias3')
hypothesis = tf.nn.tanh(tf.matmul(layer3, W4) + b4)

# cost/loss function
cost = tf.reduce_mean((Y-hypothesis)*(Y-hypothesis))

train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
            
    for step in range(320000):
        idx = step % 21; # training을 0번부터 20번까지 순차적으로 돌린다.
        sess.run(train, feed_dict={X: X_train[idx: idx + 1,:], Y: Y_train[idx:idx+1]})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X: X_train, Y: Y_train}))     
                                                                                    
    result = sess.run(hypothesis, feed_dict={X : X_test})
    result_train = sess.run(hypothesis, feed_dict={X : X_train})
    result_data = sess.run(hypothesis, feed_dict = {X : X_normdata})
    
    print("R square")
    print(r2_score(Y_normdata,result_data))
    print("result")
    print(result*(652.37031-(-447.62963))+1148.62963)
    print("Y_test")
    print(Y_test*(652.37031-(-447.62963))+1148.62963)
    print("result_train")
    print(result_train*(652.37031-(-447.62963))+1148.62963)
    print("result_data")
    print(result_data*(652.37031-(-447.62963))+1148.62963)

    real = Y_normdata*(652.37031-(-447.62963))+1148.62963
    prediction = result_data*(652.37031-(-447.62963))+1148.62963
    
    print("real")
    print(real)
    print("prediction")
    print(prediction)
    print("R square")
    print(r2_score(real,prediction))

0 0.0540544
100 0.054047
200 0.0540421
300 0.0540379
400 0.054034
500 0.0540292
600 0.0540244
700 0.0540202
800 0.0540165
900 0.0540121
1000 0.0540074
1100 0.0540032
1200 0.0539996
1300 0.0539956
1400 0.053991
1500 0.0539869
1600 0.0539833
1700 0.0539797
1800 0.0539752
1900 0.0539711
2000 0.0539676
2100 0.0539642
2200 0.0539599
2300 0.0539557
2400 0.0539522
2500 0.053949
2600 0.0539449
2700 0.0539408
2800 0.0539371
2900 0.0539339
3000 0.0539301
3100 0.053926
3200 0.0539223
3300 0.053919
3400 0.0539153
3500 0.0539111
3600 0.0539072
3700 0.0539038
3800 0.0539004
3900 0.0538961
4000 0.0538922
4100 0.0538889
4200 0.0538856
4300 0.0538814
4400 0.0538773
4500 0.053874
4600 0.0538708
4700 0.0538667
4800 0.0538626
4900 0.0538591
5000 0.0538558
5100 0.053852
5200 0.0538477
5300 0.053844
5400 0.0538407
5500 0.0538371
5600 0.053833
5700 0.0538293
5800 0.0538263
5900 0.0538232
6000 0.0538193
6100 0.0538158
6200 0.053813
6300 0.0538104
6400 0.0538068
6500 0.0538035
6600 0.0538008
6700 0.0537984
680

52400 0.0986719
52500 0.0986682
52600 0.0986789
52700 0.0986825
52800 0.0986913
52900 0.0986848
53000 0.0986967
53100 0.0987012
53200 0.0987087
53300 0.0987015
53400 0.0987119
53500 0.0987168
53600 0.0987223
53700 0.0987201
53800 0.098726
53900 0.0987292
54000 0.098733
54100 0.0987334
54200 0.098739
54300 0.0987456
54400 0.0987457
54500 0.0987542
54600 0.0987516
54700 0.0987571
54800 0.0987605
54900 0.0987697
55000 0.0987616
55100 0.0987717
55200 0.0987774
55300 0.0987837
55400 0.0987748
55500 0.098783
55600 0.0987877
55700 0.0987923
55800 0.0987871
55900 0.0987917
56000 0.0987974
56100 0.0987991
56200 0.0987969
56300 0.0988024
56400 0.098807
56500 0.0988086
56600 0.0988134
56700 0.0988088
56800 0.0988181
56900 0.0988197
57000 0.0988255
57100 0.0988176
57200 0.0988271
57300 0.0988307
57400 0.0988336
57500 0.0988266
57600 0.098833
57700 0.0988379
57800 0.0988402
57900 0.0988385
58000 0.0988427
58100 0.098847
58200 0.0988529
58300 0.0988482
58400 0.0988508
58500 0.0988551
58600 0.0988577

104000 0.0990324
104100 0.0990276
104200 0.0990253
104300 0.09903
104400 0.0990306
104500 0.099028
104600 0.0990328
104700 0.0990336
104800 0.0990349
104900 0.0990374
105000 0.0990371
105100 0.099039
105200 0.0990367
105300 0.0990466
105400 0.0990368
105500 0.0990468
105600 0.099037
105700 0.099045
105800 0.0990426
105900 0.0990477
106000 0.0990526
106100 0.099053
106200 0.0990485
106300 0.0990537
106400 0.0990527
106500 0.0990564
106600 0.0990516
106700 0.0990533
106800 0.0990585
106900 0.0990539
107000 0.0990551
107100 0.0990526
107200 0.0990621
107300 0.099057
107400 0.0990644
107500 0.0990571
107600 0.0990658
107700 0.0990638
107800 0.0990666
107900 0.0990614
108000 0.0990675
108100 0.0990685
108200 0.0990681
108300 0.0990672
108400 0.0990684
108500 0.0990719
108600 0.0990717
108700 0.0990679
108800 0.0990712
108900 0.0990761
109000 0.0990734
109100 0.0990743
109200 0.0990723
109300 0.0990777
109400 0.0990737
109500 0.0990798
109600 0.099075
109700 0.099079
109800 0.0990807
109900 

152800 0.0993588
152900 0.0993637
153000 0.0993664
153100 0.0993634
153200 0.0993702
153300 0.0993704
153400 0.0993779
153500 0.0993731
153600 0.0993821
153700 0.0993773
153800 0.0993875
153900 0.0993865
154000 0.0993916
154100 0.0993893
154200 0.0993903
154300 0.0993942
154400 0.0993994
154500 0.0993869
154600 0.0993959
154700 0.0993964
154800 0.0993959
154900 0.0993976
155000 0.0994027
155100 0.0994028
155200 0.0994012
155300 0.0994059
155400 0.0994065
155500 0.099415
155600 0.0994034
155700 0.0994197
155800 0.0994127
155900 0.0994239
156000 0.0994213
156100 0.0994324
156200 0.0994242
156300 0.0994311
156400 0.099434
156500 0.0994372
156600 0.0994386
156700 0.0994354
156800 0.0994416
156900 0.0994424
157000 0.0994426
157100 0.0994519
157200 0.0994473
157300 0.0994485
157400 0.0994526
157500 0.0994535
157600 0.0994612
157700 0.0994515
157800 0.0994618
157900 0.0994601
158000 0.0994661
158100 0.0994684
158200 0.0994664
158300 0.0994676
158400 0.0994746
158500 0.0994783
158600 0.099476


202600 0.100661
202700 0.100672
202800 0.100673
202900 0.100681
203000 0.100682
203100 0.100686
203200 0.100687
203300 0.100697
203400 0.100698
203500 0.100696
203600 0.100706
203700 0.100711
203800 0.10071
203900 0.100708
204000 0.100714
204100 0.100721
204200 0.100729
204300 0.100728
204400 0.100733
204500 0.100735
204600 0.100743
204700 0.10074
204800 0.100746
204900 0.100752
205000 0.100754
205100 0.100756
205200 0.10076
205300 0.100756
205400 0.100767
205500 0.100771
205600 0.100766
205700 0.100774
205800 0.100778
205900 0.100782
206000 0.100773
206100 0.100791
206200 0.100787
206300 0.100792
206400 0.100789
206500 0.100796
206600 0.100792
206700 0.100803
206800 0.100807
206900 0.100812
207000 0.100808
207100 0.100817
207200 0.10082
207300 0.100813
207400 0.100823
207500 0.100831
207600 0.100829
207700 0.100824
207800 0.100837
207900 0.100835
208000 0.100844
208100 0.100834
208200 0.100842
208300 0.100848
208400 0.100852
208500 0.100848
208600 0.100859
208700 0.100861
208800 0.100

254200 0.101835
254300 0.101824
254400 0.101847
254500 0.101836
254600 0.10184
254700 0.101836
254800 0.101844
254900 0.10185
255000 0.101858
255100 0.101851
255200 0.101865
255300 0.101871
255400 0.101876
255500 0.101878
255600 0.101882
255700 0.101888
255800 0.101893
255900 0.101897
256000 0.101888
256100 0.101905
256200 0.101905
256300 0.101908
256400 0.101901
256500 0.101921
256600 0.101911
256700 0.101922
256800 0.101919
256900 0.101924
257000 0.101926
257100 0.10193
257200 0.101928
257300 0.101938
257400 0.101941
257500 0.101941
257600 0.101943
257700 0.101944
257800 0.101944
257900 0.101953
258000 0.101954
258100 0.101947
258200 0.101959
258300 0.10196
258400 0.101961
258500 0.101954
258600 0.101971
258700 0.101964
258800 0.10197
258900 0.101965
259000 0.101972
259100 0.101974
259200 0.101979
259300 0.101975
259400 0.101983
259500 0.10198
259600 0.101984
259700 0.101982
259800 0.101984
259900 0.101984
260000 0.101994
260100 0.101988
260200 0.101984
260300 0.102
260400 0.101995
2

306000 0.102743
306100 0.102742
306200 0.102748
306300 0.102747
306400 0.102746
306500 0.102755
306600 0.102756
306700 0.102755
306800 0.102742
306900 0.102765
307000 0.10276
307100 0.10276
307200 0.102759
307300 0.102773
307400 0.102762
307500 0.102769
307600 0.102768
307700 0.102776
307800 0.102775
307900 0.102773
308000 0.102776
308100 0.102775
308200 0.102784
308300 0.102785
308400 0.102785
308500 0.102777
308600 0.10279
308700 0.102787
308800 0.102791
308900 0.102781
309000 0.1028
309100 0.102793
309200 0.102795
309300 0.102796
309400 0.102805
309500 0.102803
309600 0.102804
309700 0.102804
309800 0.102813
309900 0.102814
310000 0.102811
310100 0.102812
310200 0.102811
310300 0.102816
310400 0.102817
310500 0.10282
310600 0.102805
310700 0.102823
310800 0.102825
310900 0.10283
311000 0.102816
311100 0.102834
311200 0.10283
311300 0.102828
311400 0.102828
311500 0.102846
311600 0.10284
311700 0.10284
311800 0.10284
311900 0.102845
312000 0.102844
312100 0.102846
312200 0.102843
312

In [4]:

tf.set_random_seed(777)  # for reproducibility
learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, 4])
Y = tf.placeholder(tf.float32, [None,])

W1 = tf.Variable(tf.random_normal([4, 3],stddev=0.01), name='weight1')
b1 = tf.Variable(tf.random_normal([3], stddev=0.01), name='bias1')
layer1 = tf.nn.relu(tf.matmul(X, W1) + b1)

W2 = tf.Variable(tf.random_normal([3, 4],stddev=0.01), name='weight2')
b2 = tf.Variable(tf.random_normal([4], stddev=0.01), name='bias2')
layer2 = tf.nn.relu(tf.matmul(layer1, W2) + b2)

W3 = tf.Variable(tf.random_normal([4, 3],stddev=0.01), name='weight3')
b3 = tf.Variable(tf.random_normal([3], stddev=0.01), name='bias3')
layer3 = tf.nn.relu(tf.matmul(layer2, W3) + b3)

W4 = tf.Variable(tf.random_normal([3, 2],stddev=0.01), name='weight4')
b4 = tf.Variable(tf.random_normal([2], stddev=0.01), name='bias4')
layer4 = tf.nn.relu(tf.matmul(layer3, W4) + b4)

W5 = tf.Variable(tf.random_normal([2, 1],stddev = 0.01), name='weight5')
b5 = tf.Variable(tf.random_normal([1], stddev=0.01), name='bias5')
hypothesis = tf.nn.tanh(tf.matmul(layer4, W5) + b5)

# cost/loss function
cost = tf.reduce_mean((Y-hypothesis)*(Y-hypothesis))

train = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Launch graph
with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())
            
    for step in range(320000):
        idx = step % 21;
        sess.run(train, feed_dict={X: X_train[idx: idx + 1,:], Y: Y_train[idx:idx+1]})
        if step % 100 == 0:
            print(step, sess.run(cost, feed_dict={X: X_train, Y: Y_train}))                                
                                                                                    
    result = sess.run(hypothesis, feed_dict={X : X_test})
    result_train = sess.run(hypothesis, feed_dict={X : X_train})
    result_data = sess.run(hypothesis, feed_dict = {X : X_normdata})
    
    print("R square")
    print(r2_score(Y_normdata,result_data))
    print("result")
    print(result*(652.37031-(-447.62963))+1148.62963)
    print("Y_test")
    print(Y_test*(652.37031-(-447.62963))+1148.62963)
    print("result_train")
    print(result_train*(652.37031-(-447.62963))+1148.62963)
    print("result_data")
    print(result_data*(652.37031-(-447.62963))+1148.62963)

    real = Y_normdata*(652.37031-(-447.62963))+1148.62963
    prediction = result_data*(652.37031-(-447.62963))+1148.62963
    
    print("real")
    print(real)
    print("prediction")
    print(prediction)
    print("R square")
    print(r2_score(real,prediction))

0 0.0550767
100 0.0540616
200 0.0537758
300 0.0536926
400 0.0536642
500 0.0536352
600 0.0543551
700 0.0616294
800 0.0676906
900 0.0715657
1000 0.0757006
1100 0.0791477
1200 0.0810428
1300 0.0827318
1400 0.0846238
1500 0.0865464
1600 0.0875211
1700 0.0881555
1800 0.0892841
1900 0.0910439
2000 0.0912078
2100 0.0909385
2200 0.0913909
2300 0.0929097
2400 0.0929966
2500 0.0928099
2600 0.0932847
2700 0.0946568
2800 0.0955417
2900 0.0948168
3000 0.0953107
3100 0.0960813
3200 0.0971863
3300 0.0965285
3400 0.0963441
3500 0.0970356
3600 0.0981677
3700 0.0977096
3800 0.0971541
3900 0.0973812
4000 0.09855
4100 0.0981222
4200 0.0976834
4300 0.0977644
4400 0.0986818
4500 0.0988701
4600 0.0980308
4700 0.0978845
4800 0.0987298
4900 0.0990618
5000 0.0980492
5100 0.0980003
5200 0.0985864
5300 0.0989903
5400 0.0984812
5500 0.0979534
5600 0.0983454
5700 0.0991745
5800 0.0984661
5900 0.0981755
6000 0.0982389
6100 0.0990297
6200 0.0988597
6300 0.0979711
6400 0.0981193
6500 0.0989348
6600 0.0990369
6700 0.09

52200 0.0983203
52300 0.0991056
52400 0.0989116
52500 0.0980368
52600 0.0982443
52700 0.099042
52800 0.098969
52900 0.0983178
53000 0.0981414
53100 0.0989391
53200 0.0991539
53300 0.0982299
53400 0.0981553
53500 0.0987162
53600 0.0991182
53700 0.0985626
53800 0.0980692
53900 0.0984473
54000 0.0991928
54100 0.098597
54200 0.0983132
54300 0.0983383
54400 0.0991032
54500 0.0989502
54600 0.0980165
54700 0.0982202
54800 0.0990689
54900 0.0989346
55000 0.0982413
55100 0.098097
55200 0.0989125
55300 0.0991189
55400 0.0982204
55500 0.0981493
55600 0.0987122
55700 0.0991164
55800 0.0985485
55900 0.0980902
56000 0.0985607
56100 0.0993509
56200 0.0985705
56300 0.0982433
56400 0.0982978
56500 0.0990793
56600 0.0989338
56700 0.0980079
56800 0.0982151
56900 0.0990604
57000 0.0989331
57100 0.0982395
57200 0.0980965
57300 0.0989125
57400 0.0991142
57500 0.0982205
57600 0.098149
57700 0.0988662
57800 0.0991767
57900 0.0986085
58000 0.0980659
58100 0.0984315
58200 0.0992582
58300 0.0985327
58400 0.09822

103600 0.0990427
103700 0.0982169
103800 0.0981292
103900 0.0986856
104000 0.099123
104100 0.0985473
104200 0.098073
104300 0.0984536
104400 0.0992402
104500 0.0985674
104600 0.0982506
104700 0.0983213
104800 0.0991144
104900 0.0989265
105000 0.0980373
105100 0.098243
105200 0.0990619
105300 0.0989671
105400 0.0982454
105500 0.0981202
105600 0.0989402
105700 0.0991314
105800 0.0982713
105900 0.0981744
106000 0.0987351
106100 0.0991372
106200 0.0985673
106300 0.0980738
106400 0.0984516
106500 0.0992095
106600 0.0985637
106700 0.0982455
106800 0.0983168
106900 0.0991087
107000 0.0991729
107100 0.0981213
107200 0.0982693
107300 0.0991378
107400 0.0989483
107500 0.0982281
107600 0.0980879
107700 0.0989068
107800 0.0991664
107900 0.098209
108000 0.0981304
108100 0.0987086
108200 0.099113
108300 0.0985538
108400 0.0980578
108500 0.0984379
108600 0.0992242
108700 0.098553
108800 0.098235
108900 0.0983078
109000 0.0991529
109100 0.0989972
109200 0.0980419
109300 0.0982332
109400 0.0990917
1095

152300 0.0991901
152400 0.0984074
152500 0.0981137
152600 0.0985249
152700 0.0992041
152800 0.0987441
152900 0.0980989
153000 0.0983318
153100 0.0991739
153200 0.0987912
153300 0.0981724
153400 0.0981724
153500 0.0990125
153600 0.0990551
153700 0.0980736
153800 0.0981842
153900 0.0989671
154000 0.0990505
154100 0.0982199
154200 0.0981259
154300 0.0986893
154400 0.0991316
154500 0.0985455
154600 0.0980733
154700 0.0984551
154800 0.0992471
154900 0.098572
155000 0.0982452
155100 0.0983218
155200 0.0991219
155300 0.0989274
155400 0.0980372
155500 0.0982415
155600 0.0990689
155700 0.099008
155800 0.0982899
155900 0.0981356
156000 0.0989482
156100 0.099181
156200 0.0982329
156300 0.0981457
156400 0.0987218
156500 0.0991355
156600 0.0985591
156700 0.0980696
156800 0.0984499
156900 0.0992289
157000 0.0985662
157100 0.0982385
157200 0.0983159
157300 0.0991168
157400 0.0989053
157500 0.0980326
157600 0.0983188
157700 0.099177
157800 0.0989884
157900 0.0982478
158000 0.0981089
158100 0.0989306
1

200800 0.0981828
200900 0.0984559
201000 0.0991745
201100 0.0987055
201200 0.0980942
201300 0.0983341
201400 0.0991509
201500 0.0988042
201600 0.0981757
201700 0.098314
201800 0.099084
201900 0.099152
202000 0.0980817
202100 0.0981761
202200 0.0990033
202300 0.0990418
202400 0.0983818
202500 0.098029
202600 0.0986646
202700 0.099222
202800 0.0983928
202900 0.0981766
203000 0.098451
203100 0.0991705
203200 0.0986982
203300 0.098091
203400 0.0983311
203500 0.0993898
203600 0.0988611
203700 0.0981942
203800 0.0981792
203900 0.0990133
204000 0.0991031
204100 0.0980586
204200 0.0981641
204300 0.0989938
204400 0.0990402
204500 0.0983799
204600 0.0980305
204700 0.098667
204800 0.0992218
204900 0.0983955
205000 0.0981791
205100 0.0984536
205200 0.0991732
205300 0.0987796
205400 0.098116
205500 0.0983413
205600 0.099194
205700 0.0987997
205800 0.0981677
205900 0.0981729
206000 0.0990186
206100 0.0990663
206200 0.0980729
206300 0.0981807
206400 0.0989782
206500 0.099057
206600 0.0983864
206700 0

249600 0.0982875
249700 0.0991739
249800 0.0989858
249900 0.0980313
250000 0.0982229
250100 0.0990968
250200 0.0989633
250300 0.098217
250400 0.0980966
250500 0.0989331
250600 0.0991417
250700 0.0982248
250800 0.0981318
250900 0.0987218
251000 0.0991448
251100 0.0985445
251200 0.0980668
251300 0.0984501
251400 0.0991646
251500 0.0986901
251600 0.0980918
251700 0.0983355
251800 0.0991898
251900 0.0988128
252000 0.0981731
252100 0.0981862
252200 0.0990368
252300 0.0990876
252400 0.0980854
252500 0.0981912
252600 0.0990003
252700 0.0990734
252800 0.0983881
252900 0.0980547
253000 0.0986928
253100 0.0992272
253200 0.0984477
253300 0.0982079
253400 0.0984747
253500 0.0991934
253600 0.0987226
253700 0.0980997
253800 0.0983379
253900 0.0991719
254000 0.0988114
254100 0.0981726
254200 0.0981832
254300 0.099034
254400 0.0990585
254500 0.0980827
254600 0.098188
254700 0.0989225
254800 0.0991488
254900 0.0982305
255000 0.0981442
255100 0.0987161
255200 0.0991587
255300 0.0985541
255400 0.0980804


298100 0.099035
298200 0.0980593
298300 0.0982396
298400 0.0991163
298500 0.0989759
298600 0.0982181
298700 0.0981001
298800 0.0989398
298900 0.0991509
299000 0.0982273
299100 0.0981303
299200 0.0987244
299300 0.0991513
299400 0.0985475
299500 0.0980672
299600 0.0985417
299700 0.0993731
299800 0.0985975
299900 0.098228
300000 0.0983072
300100 0.0991189
300200 0.0989442
300300 0.0980119
300400 0.0982129
300500 0.0990854
300600 0.0989683
300700 0.0982145
300800 0.0980992
300900 0.0989406
301000 0.0991421
301100 0.0982292
301200 0.0981323
301300 0.098725
301400 0.0991536
301500 0.098702
301600 0.0981322
301700 0.0984769
301800 0.0993162
301900 0.0985662
302000 0.0982086
302100 0.0982958
302200 0.0991121
302300 0.0989357
302400 0.0980095
302500 0.0982115
302600 0.0990821
302700 0.0989681
302800 0.0982141
302900 0.0980992
303000 0.0989408
303100 0.0991414
303200 0.0982513
303300 0.0981804
303400 0.0987468
303500 0.0991582
303600 0.0985638
303700 0.0980585
303800 0.0984393
303900 0.0992621
3